In [1]:
import os
import hydra
import pytorch_lightning as pl
import torch
from omegaconf import OmegaConf,open_dict
import omegaconf
import shutil
import importlib

In [2]:
cfg_auto_aggl = omegaconf.OmegaConf.load('config/auto_aggl.yaml')
cfg_denoiser_data = omegaconf.OmegaConf.load('config/denoiser/data.yaml')
cfg_denoiser_encode = omegaconf.OmegaConf.load('config/denoiser/encoder.yaml')
cfg_denoiser_global_config = omegaconf.OmegaConf.load('config/denoiser/global_config.yaml')
cfg_denoiser_model = omegaconf.OmegaConf.load('config/denoiser/model.yaml')

cfg_ae_vq_vae = omegaconf.OmegaConf.load('config/ae/vq_vae.yaml')
cfg_ae_global_config = omegaconf.OmegaConf.load('config/ae/global_config.yaml')
cfg_ae_model = omegaconf.OmegaConf.load('config/ae/model.yaml')


cfg = OmegaConf.merge(
    cfg_auto_aggl,
    {"denoiser": cfg_denoiser_data},
    {"denoiser": cfg_denoiser_encode},
    {"denoiser": cfg_denoiser_global_config},
    {"denoiser": cfg_denoiser_model},
    {"ae": cfg_ae_vq_vae},
    {"ae": cfg_ae_global_config},
    {"ae": cfg_ae_model},
    #{"verifier": omegaconf.OmegaConf.load('config/verifier/global_config.yaml')},
    #{"verifier": omegaconf.OmegaConf.load('config/verifier/model.yaml')},
)

In [12]:

data_home_dir = '/data/jhahn/data/shape_dataset/'
data_type_name = 'shape'

with open_dict(cfg):
    cfg.experiment_output_path = data_home_dir+'experiment_output/'
    cfg.denoiser.data.data_val_dir=f'{data_home_dir}/pc_data/{data_type_name}/val/'
    #cfg.denoiser.ckpt_path= data_home_dir+f'output/denoiser/everyday_epoch100_bs64/training/last.ckpt'
    cfg.denoiser.ckpt_path= data_home_dir+f'output/denoiser/shape_epoch10/training/last.ckpt'
    cfg.inference_dir= data_home_dir+'results'
    cfg.denoiser.data.val_batch_size=1
    cfg.verifier.max_iters = 1
    #cfg.verifier.ckpt_path= '/disk2/data/breaking-bad-dataset/output/verifier/everyday_epoch100_bs64/training/last.ckpt'

#print(OmegaConf.to_yaml(cfg))
#print(cfg.experiment_output_path)

In [18]:
# fix the seed
pl.seed_everything(cfg.test_seed, workers=True)

# create directories for inference outputs
inference_dir = os.path.join(cfg.experiment_output_path, "inference", cfg.inference_dir)
os.makedirs(inference_dir, exist_ok=True)

# initialize data
import jahn_src.data_loader_jhahn
import jahn_src.slice_util
importlib.reload(jahn_src.slice_util)
importlib.reload(jahn_src.data_loader_jhahn)

from jahn_src.data_loader_jhahn import build_test_dataloader


#from puzzlefusion_plusplus.denoiser.dataset.dataset import build_test_dataloader
#cfg.denoiser.data.matching_data_path= '/disk2/data/breaking-bad-dataset/data/matching_data'

test_loader = build_test_dataloader(cfg.denoiser)

Seed set to 123
100%|██████████| 1/1 [00:00<00:00, 143.08it/s]


In [19]:


import jahn_src.auto_agg_jhahn

import jahn_src.verifier_jhahn
import utils.node_merge_utils

importlib.reload(utils.node_merge_utils)
importlib.reload(jahn_src.verifier_jhahn)
importlib.reload(jahn_src.myutil)
importlib.reload(jahn_src.auto_agg_jhahn)

from jahn_src.auto_agg_jhahn import AutoAgglomerative

#import importlib
#import puzzlefusion_plusplus.auto_aggl
#importlib.reload(puzzlefusion_plusplus.auto_aggl)
#from puzzlefusion_plusplus.auto_aggl import AutoAgglomerative
# 
# 
shutil.rmtree('/data/jhahn/data/shape_dataset/results/')

model = AutoAgglomerative(cfg)

denoiser_weights = torch.load(cfg.denoiser.ckpt_path)['state_dict']

model.denoiser.load_state_dict(
    {k.replace('denoiser.', ''): v for k, v in denoiser_weights.items() 
     if k.startswith('denoiser.')}
)

model.encoder.load_state_dict(
    {k.replace('encoder.', ''): v for k, v in denoiser_weights.items() 
     if k.startswith('encoder.')}
)

# load verifier weights    
#verifier_weights = torch.load(cfg.verifier.ckpt_path)['state_dict']
#model.verifier.load_state_dict({k.replace('verifier.', ''): v for k, v in verifier_weights.items()})



trainer = pl.Trainer(accelerator=cfg.accelerator, max_epochs=1, logger=False)
trainer.test(model=model, dataloaders=test_loader)


/tmp/ipykernel_830848/3889905702.py:23: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  denoiser_weights = torch.load(cfg.denoiser.ckpt_path)['state_dict']
GPU available: True

Testing: |          | 0/? [00:00<?, ?it/s]

tensor([0.2793, 0.4031, 0.7347])
tensor([-0.2669, -0.3621, -0.7265], dtype=torch.float64)
tensor([0.0293, 0.0000, 0.9996, 0.0000], dtype=torch.float64)
tensor([0.7999, 0.3971, 0.7544])
tensor([-0.7399, -0.7231, -0.7836], dtype=torch.float64)
tensor([0.4949, 0.0000, 0.8690, 0.0000], dtype=torch.float64)
tensor([0.4388, 0.6387, 0.5247])
tensor([-0.4180, -0.7187, -0.5099], dtype=torch.float64)
tensor([0.5638, 0.0000, 0.8259, 0.0000], dtype=torch.float64)
tensor([0.3051, 0.4635, 0.4550])
tensor([-0.2986, -0.2972, -0.4439], dtype=torch.float64)
tensor([0.4968, 0.0000, 0.8679, 0.0000], dtype=torch.float64)
tensor([0.4980, 0.9371, 0.6556])
tensor([-0.6657, -0.5241, -0.5381], dtype=torch.float64)
tensor([0.2994, 0.0000, 0.9541, 0.0000], dtype=torch.float64)
test_denoiser_only
tensor([[[ 0.0000,  0.0000,  0.7347,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.5510,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000],

[{'eval/part_acc': 0.0,
  'eval/rmse_t': 0.571937084197998,
  'eval/rmse_r': 88.90299224853516,
  'eval/shape_cd': 258.4972839355469}]